In [34]:
import pandas as pd

In [35]:
gold_df = pd.read_csv('gold_dataset.csv', sep='\t')
gold_df.drop(columns=['Unnamed: 0.1'], inplace=True)
gold_df.rename(columns={'Unnamed: 0': 'gold_idx'}, inplace=True)
gold_df

,gold_idx,source,annot
0,0,Как сказал Тарас Шевченко – «нам свое делать».,Как сказал Тарас Шевченко – «нам свое делать».
1,1,"Ожидается, что новый вице-мэр по ЖКХ будет наз...","Ожидается, что новый вице-мэр по ЖКХ будет наз..."
2,2,Министерство иностранных дел Болгарии отказыва...,Министерство <abbr: иностранных |ин.> дел Болг...
3,3,Одной из причин могло послужить наблюдаемое в ...,Одной из причин могло послужить наблюдаемое в ...
4,4,"Неискушенному избирателю, «электорату» вдалбли...","Неискушенному избирателю, «электорату» вдалбли..."
...,...,...,...
110888,110888,"В другие места возвращаются, в основном, старики.","В <abbr: др.|другие> места возвращаются, в осн..."
110889,110889,Тогда погибли 8 человек и 30 получили ранения.,Тогда погибли 8 <abbr: ч.|человек> и 30 получи...
110890,110890,"Когда я была ребенком, мы стояли в очередях, ч...","Когда я была ребенком, мы стояли в очередях, ч..."
110891,110891,"Мы рассчитывали на этот матч, но его же и боял...","Мы рассчитывали на этот матч, но его же и боял..."


In [36]:
auto_df = pd.read_csv('fifth_part.csv', sep=';', encoding='cp1251')
auto_df.drop(columns=['Unnamed: 0', 'preanot'], inplace=True)
auto_df.rename(columns={'Unnamed: 4': 'eval_annot'}, inplace=True)
auto_df

,idx,text,eval_annot
0,27477,Дом Циолковского стоит на окраине Калуги.,Дом Циолковского стоит на окраине Калуги.
1,74947,"Если же вы соберётесь продавать акции, то прос...","Если же вы соберётесь продавать акции, то прос..."
2,95718,Российская девелоперская компания Mirax Group ...,Российская девелоперская компания Mirax Group ...
3,554,"Нормальная комедия, которую вполне можно посмо...","Нормальная комедия, которую вполне можно посмо..."
4,88860,"Я смирюсь с тем, что радикалам с обеих сторон ...","Я смирюсь с тем, что радикалам с обеих сторон ..."
...,...,...,...
109997,71312,Либо же он начнет укрепляться слишком быстро.,NaN
109998,101874,Не решайте чужих проблем.,NaN
109999,11307,"Главный тренер Вячеслав Быков считает, что пер...",NaN
110000,3592,Домбровский был избран председателем КС 19 сен...,NaN


In [37]:
auto_df.dropna().shape, auto_df['eval_annot'].dropna().shape

((22000, 3), (22000,))

Оказывается, всего для 22 000 строк из ~110 000 у нас есть та разметка, которую мы хотим сравнить с разметкой из `gold_df`

In [38]:
def safe_int(string: str) -> int:
    try:
        result = int(string)
    except ValueError:
        result = -1
    return result

auto_df['idx'] = auto_df['idx'].apply(safe_int)
gold_df.merge(auto_df, left_on='gold_idx', right_on='idx', how='inner').shape, \
gold_df.merge(auto_df.dropna(), left_on='gold_idx', right_on='idx', how='inner').shape

((110000, 6), (22000, 6))

Я ожидал увидеть в `auto_df` только 10 000 индексов, которые соответствовали бы индексам из `gold_df`, но почти все они соотносятся с некоторым размеченным примером из `gold_df`. В размеченных 22 000 тоже каждая строка `idx`, для которого найдется строка в `gold_df`. Значит будем присоединять датафреймы по стобцам `sample` и `text`, соответственно.

In [39]:
# замечал в файле какие-то ассиметричные последовательности кавычек (три подряд)
# на краях текстов, так что на всякий случай для лучшего сравнивания обрежем их
def safe_strip(string: str) -> str:
    if isinstance(string, str):
        result = string.strip('" ')
    else:
        print(string, type(string))
        result = string
    return result

In [40]:
auto_df['text_stripped'] = auto_df['text'].apply(safe_strip)
gold_df['source_stripped'] = gold_df['source'].apply(safe_strip)

nan <class 'float'>


In [41]:
auto_df.dropna(inplace=True)
gold_df.dropna(inplace=True)

In [42]:
eval_df = auto_df.merge(gold_df, left_on='text_stripped', right_on='source_stripped')
eval_df.drop(columns=['gold_idx', 'source', 'source_stripped', 'text_stripped'], inplace=True)
eval_df = eval_df[eval_df['eval_annot'] != eval_df['annot']]

In [52]:
del auto_df, gold_df
eval_df

,idx,text,eval_annot,annot,eval_annot_abbrs,annot_abbrs
4,4784,При покупке в кредит недвижимости на первичном...,При покупке в кредит недвижимости на первичном...,При покупке в кредит недвижимости на первичном...,[],[]
5,4964,В городе пряников и самоваров страховать пасса...,В <abbr: гор.|городе> пряников и самоваров стр...,В <abbr: гор.|городе> пряников и самоваров стр...,[],[]
7,2009,Еще в 80-х годах его толщина достигала полутор...,Еще в 80-х <abbr: гг.|годах> его толщина дости...,Еще в 80-х <abbr: годах |г.> его толщина дости...,[],[]
8,2715,В этой беседке к потолку подвешена кованая люс...,В этой беседке к потолку подвешена кованая люс...,В этой беседке к потолку подвешена кованая люс...,[],[]
9,2760,"Из практики знаю, что просто так недовольство ...","Из практики знаю, что просто так недовольство ...","Из практики знаю, что просто так недовольство ...",[],[]
...,...,...,...,...,...,...
1718,6717,"Однако, услышав ее в исполнении омоновца, я по...","Однако, услышав ее в исполнении омоновца, я по...","Однако, услышав ее в исполнении омоновца, я по...",[],[]
1719,2473,С ноября 1990 года по ноябрь 1992 года проходи...,С ноября 1990 <abbr: г.|года> по ноябрь 1992 <...,С ноября 1990 <abbr: года |г.> по ноябрь 1992 ...,[],[]
1724,9071,"Мы не можем утверждать, что младенцы рождаются...","Мы не можем утверждать, что младенцы рождаются...","Мы не можем утверждать, что младенцы рождаются...",[],[]
1725,9493,Ведь для русского бильярда и кий требуется спе...,Ведь для <abbr: рус.|русского> бильярда и кий ...,Ведь для <abbr: рус.| русского > бильярда и ки...,[],[]


In [45]:
eval_df.to_csv('eval.csv', sep='\t', index=False)

Я так понял смысл разметки, которую мы хотим получить: если мы встречаем сокращение, то заменяем его строкой вида "<abbr: сокращение|сокр.>", а если встречаем что-то сокращаемое, то заменяем его на "<abbr: сокр.|сокращение>". Тогда имеет смысл посмотреть на то, как часто у нас встречаются такие упорядоченные пары в каждом виде разметки.

In [85]:
import re

abbr_to_full = {'gold': dict(), 'eval': dict()}  # случаи раскрытия аббревиатур
full_to_abbr = {'gold': dict(), 'eval': dict()}  # случая сведения к аббревиатуре

def count_abbrs(string: str, df: str) -> list:

    result = re.findall(r'<[^<>]+>', string)  # находим выражения вида "<abbr: тыс.|тысяч>"
    result = list(map(lambda x: x[7:-1].strip(), result))  # обрезаем до "тыс.|тысяч"
    result = list(map(lambda x: tuple((token.strip() for token in x.split('|'))), result))  # делим на два слова и обрезаем пробелы
    # result = list(map(lambda x: )

    for pair in result:
        # понятно, не идеальный критерий, но кое-что наглядное мы получим
        if len(pair[0].strip('.')) <= len(pair[1].strip('.')):  # <abbr: тыс.|тысяч>
            abbrs = full_to_abbr
        else:  # <abbr: тысяч|тыс.>
            abbrs = abbr_to_full

        if pair not in abbrs[df]:
            abbrs['gold'][pair] = 0
            abbrs['eval'][pair] = 0
        abbrs[df][pair] += 1

    return result

In [79]:
%%capture
eval_df['eval_annot'].apply(lambda x: count_abbrs(x, 'eval'))
eval_df['annot'].apply(lambda x: count_abbrs(x, 'gold'))

# eval_df['eval_annot_abbrs'] = eval_df['eval_annot'].apply(lambda x: count_abbrs(x, 'eval'))
# eval_df['annot_abbrs'] = eval_df['annot'].apply(lambda x: count_abbrs(x, 'gold'))

In [82]:
# статистика по сведению слов к аббревиатурам
print('gold eval - сокращение|сокр.')
for pair, _ in sorted(abbr_to_full['gold'].items(), key=lambda x: x[1], reverse=True):
    print(f'{abbr_to_full["gold"][pair]: <4} {abbr_to_full["eval"][pair]: <4} - {pair[0]}|{pair[1]}')

gold eval - сокращение|сокр.
35   0    - года|г.
35   0    - года|гг.
22   0    - году|г.
22   0    - году|гг.
17   1    - доллар|$
17   1    - доллар|долл.
16   0    - гор|г.
16   0    - гор|гг.
11   0    - месяц|мес.
11   0    - месяц|м-ц
4    0    - годах|г.
4    0    - годах|гг.
4    0    - год|г.
4    0    - год|гг.
4    0    - годы|г.
4    0    - годы|гг.
3    0    - народный артист|н. а.
3    0    - народный артист|нар. арт
3    0    - месяца|мес.
3    0    - месяца|м-ц
3    0    - воскресенье|воскр.
3    0    - воскресенье|вс.
2    0    - иностранных|ин.
2    0    - иностранных|иностр.
2    0    - долларов|$
2    0    - долларов|долл.
2    0    - годам|г.
2    0    - годам|гг.
2    0    - белорусские|белорус.
2    0    - белорусские|блр.
1    0    - так называют|т. наз.
1    0    - так называют|так наз.
1    0    - страниц|с.
1    0    - страниц|стр.
1    0    - месяцев|мес.
1    0    - месяцев|м-ц
1    0    - так называемой|т. наз.
1    0    - так называемой|так наз.
1    0   

In [84]:
# статистика по раскрытию аббревиатур
print('gold eval - сокращение|сокр.')
for pair, _ in sorted(full_to_abbr['gold'].items(), key=lambda x: x[1], reverse=True):
    print(f'{full_to_abbr["gold"][pair]: <4} {full_to_abbr["eval"][pair]: <4} - {pair[0]}|{pair[1]}')

gold eval - сокращение|сокр.
41   11   - %|процент
39   0    - т.|то
26   35   - т.|тысяч
18   27   - ч.|человек
15   16   - д.|день
13   8    - нач.|начала
12   33   - млн|миллионов
12   5    - млн|миллион
10   93   - г.|года
10   14   - млн|миллиона
9    0    - т.|тем
9    0    - |доллар
9    0    - олл.|доллар
8    5    - гор.|городе
8    6    - г.|городе
8    8    - гл.|глава
8    7    - г.|города
8    7    - гор.|города
8    9    - обл.|области
8    7    - д.|дома
8    5    - ч.|числе
7    7    - ч.|человека
7    7    - г.|город
7    8    - ц.|центре
7    4    - гор.|город
6    6    - д.|дней
6    6    - кв.|квартале
6    8    - д.|дня
6    5    - др.|другие
6    13   - млрд|миллиардов
6    2    - нач.|начале
5    56   - %|процентов
5    38   - тыс.|тысяч
5    38   - г.|году
5    5    - ч.|части
5    4    - пт.|пятницу
5    5    - ч.|часть
5    0    - т.|тот
5    0    - т.|те
4    20   - %|процента
4    4    - росс.|российские
4    3    - гл.|главы
4    3    - ф.|фунтов
4    5    

In [49]:
# import re
#
# abbrs = {'gold': dict(), 'eval': dict()}  # тут храним информацию о частоте встречаемости
#
# def count_abbrs(string: str, df: str) -> list:
#
#     result = re.findall(r'<[^<>]+>', string)  # находим выражения вида "<abbr: тыс.|тысяч>"
#     result = list(map(lambda x: x[7:-1].strip(), result))  # обрезаем до "тыс. | тысяч"
#     result = list(map(lambda x: tuple(token.strip() for token in x.split('|')), result))  # делим на два слова и обрезаем пробелы
#     result = list(map(lambda x: map())) # своидим случаи тыс
#
#     for abbr in result:
#         if abbr not in abbrs[df]:
#             abbrs['gold'][abbr] = 0
#             abbrs['eval'][abbr] = 0
#         abbrs[df][abbr] += 1
#
#     return result